# Демченко Георгий Павлович БПИ-235, Task02_Part02

### Установка данных и подключение библиотек

In [1]:
# Ссылка на папку яндекс диска, где располагаются необходимые данные - "https://disk.yandex.ru/d/qgR2RTVMlfWmvg"
import requests
from urllib.parse import urlencode

url_to_data = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?' + urlencode(dict(public_key = 'https://disk.yandex.ru/d/qgR2RTVMlfWmvg'))
response = requests.get(url_to_data)
download_url = response.json()['href']

download_response = requests.get(download_url)
with open('GEO_32637.zip', 'wb') as f:
    f.write(download_response.content)

In [ ]:
import geopandas, pandas, numpy, shapely
!unzip -o GEO_32637.zip
%pip install -U mapclassify
import mapclassify
from google.colab import files

## Задание №1 - "Городское планирование"

In [3]:
# @title Сперва загрузим необходимые объекты и создадим для них GDF
def Get_GDF(obj_url):
  particle_url = "GEO_32637/DATA_32637/"
  return geopandas.read_file(f"{particle_url}{obj_url}")

# Файл с созданным Linestring объектом в формате ESRI Shapefile был создан в QGIS и загружен в папку, откуда берём данные
line_road_df = Get_GDF("road.shp")
railway_station_df = Get_GDF("railway-station-point_32637.shp")
buildings_poly_df = Get_GDF("building-polygon_32637.shp")

In [4]:
# @title Теперь найдем остановки и здания, которые пересекаются с дорогой, ширина дороги - 20 метров
# Сперва расширим дорогу до 20 метров в ширину
line_road_df['geometry'] = numpy.where(line_road_df["NAME"] == "KG_to_HSE", line_road_df.geometry.buffer(20), line_road_df.geometry)

In [5]:
#Найдем искомые объекты
buildings_demolish_df = buildings_poly_df.sjoin(line_road_df, how = "inner", predicate = "intersects")
railway_demolish_df = railway_station_df.sjoin(line_road_df, how = "inner", predicate = "intersects")

In [ ]:
# @title Можем посмoтреть как выглядят полученные объекты на карте
# 1
line_road_df.explore()
# 2
# buildings_demolish_df.explore()
# 3
# railway_demolish_df.explore()

### Скачаем полученные в задании №1 GDF в формате ESRI Shapefile

In [ ]:
# Сохранение данных о зданиях, которые необходимо снести
buildings_demolish_df.to_file("building-demolish.shp", mode = "w", crs = "EPSG:32637", driver='ESRI Shapefile', encoding = "UTF-8")
files.download("building-demolish.shp")

In [ ]:
buildings_demolish_df.to_file("building-demolish.dbf", mode = "w", crs = "EPSG:32637", driver='ESRI Shapefile', encoding = "UTF-8")
files.download("building-demolish.dbf")

In [ ]:
buildings_demolish_df.to_file("building-demolish.shx", mode = "w", crs = "EPSG:32637", driver='ESRI Shapefile', encoding = "UTF-8")
files.download("building-demolish.shx")

In [ ]:
# Сохранение данных о станциях, которые необходимо снести
railway_demolish_df.to_file("railway-demolish.shp", mode = "w", crs = "EPSG:32637", driver='ESRI Shapefile', encoding = "UTF-8")
files.download("railway-demolish.shp")

In [ ]:
railway_demolish_df.to_file("railway-demolish.dbf", mode = "w", crs = "EPSG:32637", driver='ESRI Shapefile', encoding = "UTF-8")
files.download("railway-demolish.dbf")

In [ ]:
railway_demolish_df.to_file("railway-demolish.shx", mode = "w", crs = "EPSG:32637", driver='ESRI Shapefile', encoding = "UTF-8")
files.download("railway-demolish.shx")

## Задание №2 - "Доступность магазинов"


In [16]:
# @title Сперва загрузим необходимые объекты и создадим для них GDF
poi_point_df = Get_GDF("poi-point_32637.shp")
railway_station_df = Get_GDF("railway-station-point_32637.shp")

In [17]:
# @title Проведём анализ полученных данных и установим какие обьекты нам нужны
poi_point_df["SHOP"].unique()
lst_unacceptable_shops = [None, "null"]
railway_station_df["RAILWAY"].unique()
lst_acceptable_stations = ["tram_stop"]

In [18]:
# @title Создадим необходимые объекты на основе анализа
shop_points_df = poi_point_df[~(poi_point_df["SHOP"].isin(lst_unacceptable_shops) == True)]
tram_stop_df = railway_station_df[railway_station_df["RAILWAY"].isin(lst_acceptable_stations) == True]
tram_stop_area_df = tram_stop_df.copy()
tram_stop_area_df["geometry"] = numpy.where(tram_stop_area_df["RAILWAY"] == "tram_stop", tram_stop_area_df.geometry.buffer(200), tram_stop_area_df.geometry)
# Создадим GDF с информацией о магазинах, расположенных в пределе 200 метров от трамваной остановки
shops_near_tram_stops_df = shop_points_df.sjoin(tram_stop_area_df, how = "inner", predicate = "intersects")
shops_near_tram_stops_df.head()

,OSM_ID_left,NAME_left,MAN_MADE,LEISURE,AMENITY,OFFICE,SHOP,TOURISM,SPORT,geometry,index_right,OSM_ID_right,NAME_right,RAILWAY
2226,7.126169e+08,None,None,None,None,None,hairdresser,None,None,POINT (399193.133 6185127.238),37,946193043.0,Улица Кулакова,tram_stop
2226,7.126169e+08,None,None,None,None,None,hairdresser,None,None,POINT (399193.133 6185127.238),36,834436582.0,Улица Кулакова,tram_stop
2226,7.126169e+08,None,None,None,None,None,hairdresser,None,None,POINT (399193.133 6185127.238),34,944665395.0,Таллинская улица,tram_stop
2226,7.126169e+08,None,None,None,None,None,hairdresser,None,None,POINT (399193.133 6185127.238),35,250559604.0,Таллинская улица,tram_stop
2304,1.119409e+09,М.Видео,None,None,None,None,electronics,None,None,POINT (399415.838 6185651.527),39,944656870.0,Строгинский бульвар,tram_stop


In [ ]:
# @title Создадим требуемый GDF с Linestring объектами на основе полуеных данных
# Определим дополнительные функции
def Get_TramStop_Name_OSM_Id(id):
  name_cur = [x for x in tram_stop_df[tram_stop_df["OSM_ID"] == id]["NAME"]][0]
  if (name_cur == None):
    return "N/a"
  return name_cur

def Get_Tram_Stop_Geometry_OSM_Id(id):
  geometry_cur = [x for x in tram_stop_df[tram_stop_df["OSM_ID"] == id]["geometry"]][0]
  return geometry_cur

# GDF с информацией о парах магазин - остановка уже был создан в предыдущеи пункте , осталось создать GDF c LineString объектами на основе этх пар
lst_linestring_wkts = []
data_for_LineString = {"SHOP_NAME": [],
                          "TRAM_S_N": [],
                          "SHOP_TYPE": [],
                          "DISTANCE": []}

for idx in range(len(shops_near_tram_stops_df)):
  if idx % 100 == 0:
    print(round(idx / len(shops_near_tram_stops_df) * 100, 2), '%')
  cur_shop_name_tmp = [x for x in shops_near_tram_stops_df.iloc[[idx]]["NAME_left"]][0]
  cur_shop_name = cur_shop_name_tmp if (cur_shop_name_tmp != None) else "N/a"
  cur_shop_type = [x for x in shops_near_tram_stops_df.iloc[[idx]]["SHOP"]][0]
  cur_tram_osm_id = [x for x in shops_near_tram_stops_df.iloc[[idx]]["OSM_ID_right"]][0]
  cur_shop_point_geometry = [x for x in shops_near_tram_stops_df.iloc[[idx]]["geometry"]][0]
  cur_tram_stop_name = Get_TramStop_Name_OSM_Id(cur_tram_osm_id)
  cur_tram_stop_point_geometry = Get_Tram_Stop_Geometry_OSM_Id(cur_tram_osm_id)
  cur_linestring = shapely.LineString([cur_shop_point_geometry, cur_tram_stop_point_geometry]).wkt
  data_for_LineString["SHOP_NAME"].append(cur_shop_name)
  data_for_LineString["TRAM_S_N"].append(cur_tram_stop_name)
  data_for_LineString["SHOP_TYPE"].append(cur_shop_type)
  lst_linestring_wkts.append(cur_linestring)

geom_linestring = geopandas.GeoSeries.from_wkt(lst_linestring_wkts, crs = 32637)
data_for_LineString["DISTANCE"] = [f"{round(x.length, 2)} m." for x in geom_linestring]
tram_shops_paths_df = geopandas.GeoDataFrame(data = data_for_LineString, geometry = geom_linestring)
tram_shops_paths_df.head()

In [ ]:
#@title Можно посмотреть как выглядят объекты из полученного GDF на карте
tram_shops_paths_df.explore()

### Скачаем полученные в задании №2 GDF в формате ESRI Shapefile

In [21]:
# Сохранение данных о сегметнах
tram_shops_paths_df.to_file("segments.shp", mode = "w", crs = "EPSG:32637", driver='ESRI Shapefile', encoding = "UTF-8")
files.download("segments.shp")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [22]:
tram_shops_paths_df.to_file("segments.dbf", mode = "w", crs = "EPSG:32637", driver='ESRI Shapefile', encoding = "UTF-8")
files.download("segments.dbf")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [23]:
tram_shops_paths_df.to_file("segments.shx", mode = "w", crs = "EPSG:32637", driver='ESRI Shapefile', encoding = "UTF-8")
files.download("segments.shx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Сохранение данных о трамвайных остановках
tram_stop_df.to_file("tram_stop_poi.shp", mode = "w", crs = "EPSG:32637", driver='ESRI Shapefile', encoding = "UTF-8")
files.download("tram_stop_poi.shp")

In [ ]:
tram_stop_df.to_file("tram_stop_poi.dbf", mode = "w", crs = "EPSG:32637", driver='ESRI Shapefile', encoding = "UTF-8")
files.download("tram_stop_poi.dbf")

In [ ]:
tram_stop_df.to_file("tram_stop_poi.shx", mode = "w", crs = "EPSG:32637", driver='ESRI Shapefile', encoding = "UTF-8")
files.download("tram_stop_poi.shx")

In [ ]:
# Сохранение данных о магазинах вблизи трамвайных остановок
shops_near_tram_stops_df.to_file("shop_near_stops_poi.shp", mode = "w", driver='ESRI Shapefile', crs = "EPSG:32637", encoding = "UTF-8")
files.download("shop_near_stops_poi.shp")

In [ ]:
shops_near_tram_stops_df.to_file("shop_near_stops_poi.dbf", mode = "w", driver='ESRI Shapefile', crs = "EPSG:32637", encoding = "UTF-8")
files.download("shop_near_stops_poi.dbf")

In [ ]:
shops_near_tram_stops_df.to_file("shop_near_stops_poi.shx", mode = "w", driver='ESRI Shapefile', crs = "EPSG:32637", encoding = "UTF-8")
files.download("shop_near_stops_poi.shx")